In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import mlflow.sklearn

from data.data_source import get_data_source
from data.treasury_curve import get_yield_curve
# from models.covariance.empirical_covariance import EmpiricalCovarianceModel as model_choice
from models.covariance.garch import GARCHCovarianceModel as model_choice
from config import env
from utils.artifact_saver import get_artifact_path
import math
import warnings
from arch.univariate.base import DataScaleWarning
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=DataScaleWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# ─── CONFIG ─────────────────────────────────────────────────────────────────
CURVE_TYPE        = "US Treasury Par"
TENORS            = [1/12, 0.125, 2/12, 0.25, 4/12, 0.5, 1, 2, 3, 5, 7, 10, 20, 30]
N_SIMS            = 1000
MLFLOW_EXPERIMENT = "IR Cone Fit Experiment"
backfill_DAYS     = 180       # how many days back to pull data
MAX_WORKERS       = 12
ds                = get_data_source()
model_class = model_choice
model_shortname = model_class.name


def format_tenor(x):
    total_months = round(x * 12 * 2) / 2
    years = int(total_months // 12)
    months = total_months - years * 12
    parts = []
    if years:
        parts.append(f"{years}Y")
    if months:
        parts.append(f"{months:.1f}M" if not months.is_integer() else f"{int(months)}M")
    return "".join(parts) or "0M"


def batch_insert_rate_cones(records, batch_size=200):
    total = 0
    for i in range(0, len(records), batch_size):
        batch = records[i : i + batch_size]
        vals = ",\n".join(
            f"('{r['curve_type']}', {r['days_forward']:.1f}, '{r['curve_date']}', "
            f"'{r['cone_type']}', '{r['tenor_str']}', {r['rate']:.8f}, {r['tenor_num']:.8f}, '{r['model_type']}')"
            for r in batch
        )        
        sql = f"""
        INSERT INTO rate_cones
          (curve_type, days_forward, curve_date, cone_type, tenor_str, rate, tenor_num, model_type)
        VALUES {vals}
        ON CONFLICT DO NOTHING;
        """
        ds.query(sql)
        total += len(batch)
    return total


def plot_ir_cones_matplotlib(base_curve: pd.Series, ir_cone_df: pd.DataFrame, days_forward: int, title: str = ""):
    plt.figure(figsize=(10, 6))
    sample_ids = np.random.choice(
        ir_cone_df["sim_id"].unique(),
        size=min(100, ir_cone_df["sim_id"].nunique()),
        replace=False
    )
    for sim_id in sample_ids:
        sim = ir_cone_df[ir_cone_df["sim_id"] == sim_id]
        plt.plot(sim["tenor_num"], sim["rate_simulated"], color="gray", alpha=0.1)

    plt.plot(base_curve.index, base_curve.values,
             color="crimson", linewidth=2.5, label="Base Curve")
    plt.xlabel("Tenor (years)")
    plt.ylabel("Yield (%)")
    plt.title(title or f"{days_forward}-Day IR Cones ({N_SIMS} sims) on {base_curve.name}")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    fn = get_artifact_path(f"tsy_cones_{base_curve.name}_{days_forward}d.png")
    plt.savefig(fn, dpi=150)
    plt.close()
    return fn


def generate_ir_cone(base_curve: pd.Series,
                     cov_model: model_class,
                     n_sims: int,
                     days_forward: int) -> pd.DataFrame:

    N = len(base_curve)
    
    cov = cov_model.covariance_ * days_forward
    
    drift = getattr(cov_model, "drift_", np.zeros(N))
    drift_fw = drift * days_forward  # horizon drift    

    rand_deltas = np.random.multivariate_normal(
        mean=np.zeros(len(base_curve)),
        cov=cov,
        size=n_sims
    )
    
    base_vals = base_curve.to_numpy()
    sims = base_vals[np.newaxis, :] + drift_fw[np.newaxis, :] + rand_deltas

    records = [
        {"sim_id": sim, "tenor_num": base_curve.index[i], "rate_simulated": sims[sim, i]}
        for sim in range(n_sims)
        for i in range(len(base_curve))
    ]
    return pd.DataFrame.from_records(records)


def populate_ir_cones(backfill_days: int,
                      fit_window_years: int = 1,
                      years_back: int = 0,
                      max_workers: int = 4):
    end_date   = datetime.today().date()
    start_date = max(
        end_date - relativedelta(days=backfill_days, years=years_back),
        datetime(2010, 1, 1).date()
    )
    all_dates = pd.date_range(start=start_date, end=end_date, freq="D").date

    mlflow.set_experiment(MLFLOW_EXPERIMENT)
    client = MlflowClient()

    with mlflow.start_run(run_name=f"populate_ir_cones_{end_date}") as parent:
        parent_id = parent.info.run_id
        mlflow.log_params({
            "as_of_date": str(start_date),
            "backfill_days": backfill_days,
            "fit_window_years": fit_window_years,
            "curve_type": CURVE_TYPE,
            "n_sims": N_SIMS,
        })

        total_obs, total_vars, trace_covs, errors = [], [], [], []

        def task(asof_date):
            try:
                window_start = asof_date - relativedelta(years=fit_window_years)
                window_start = max(window_start, datetime(2010,1,1).date())

                sql = f"""
                SELECT curve_date, tenor_num, rate
                  FROM rate_curves
                 WHERE curve_type = '{CURVE_TYPE}'
                   AND curve_date BETWEEN '{window_start}' AND '{asof_date}'
                   AND tenor_num    IN ({', '.join(map(str, TENORS))})
                 ORDER BY curve_date DESC, tenor_num;
                """
                df = ds.query(sql).to_pandas()
                if df.empty:
                    return (asof_date, "No curve data")

                pivot = (
                    df.pivot(index="curve_date", columns="tenor_num", values="rate")
                      .sort_index()
                      .interpolate(method="linear", axis=0)
                      .dropna()
                )
                if asof_date not in pivot.index:
                    print(f"⏭️  Skipping {asof_date}: no exact curve_date in pivot")
                    return None

                asof_actual = (asof_date if asof_date in pivot.index
                               else pivot.index[pivot.index <= asof_date].max())
                base_curve = pivot.loc[asof_date]
                deltas     = pivot.diff().dropna()   # these deltas now span your lookback window

                model_name = f'{model_shortname}_{fit_window_years}yrFit'
                print('using model: ' + model_name)

                for days_forward in (30, 90):
                    model   = model_class().fit(deltas.values)
                    cone_df = generate_ir_cone(base_curve, model, N_SIMS, days_forward)
                    chart   = plot_ir_cones_matplotlib(base_curve, cone_df,
                                                       days_forward,
                                                       title=f"{days_forward}-day cones")
    
                    pctls = [1,5,10,50,90,95,99]
                    pct_df = (
                        cone_df.groupby("tenor_num")["rate_simulated"]
                               .quantile([p/100 for p in pctls])
                               .unstack(level=1)
                               .reset_index()
                               .melt(id_vars="tenor_num", var_name="percentile", value_name="rate")
                    )
                    pct_df["percentile"] = pct_df["percentile"].astype(float)
    
                    pct_df["curve_type"]   = CURVE_TYPE
                    pct_df["tenor_str"]    = pct_df["tenor_num"].apply(format_tenor)
                    pct_df["cone_type"]    = pct_df["percentile"].apply(lambda p: f"{int(p*100)}%")
                    pct_df["curve_date"]   = asof_date
                    pct_df["days_forward"] = days_forward
                    pct_df["model_type"] = model_name
    
                    recs = pct_df[[
                        "curve_type","days_forward","curve_date",
                        "cone_type","tenor_str","rate","tenor_num", "model_type"
                    ]].to_dict(orient="records")
                    inserted = batch_insert_rate_cones(recs)
    
                    n_obs     = len(deltas)
                    total_var = float(np.var(deltas.values))
                    trace_cv  = float(np.trace(model.covariance_))
    
                    total_obs.append(n_obs)
                    total_vars.append(total_var)
                    trace_covs.append(trace_cv)
                
                    input_example = deltas.values[:1]
    
                    with mlflow.start_run(
                        run_name=f"IR_{asof_date}",
                        nested=True,
                        tags={"mlflow.parentRunId": parent_id}
                    ):
                        mlflow.log_params({
                            "as_of_date": str(asof_actual),
                            "backfill_days": backfill_days,
                            "fit_window_years": fit_window_years,
                            "curve_type": CURVE_TYPE,
                            "n_sims": N_SIMS,
                        })
                        mlflow.log_metrics({
                            "n_obs": n_obs,
                            "total_var": total_var,
                            "trace_cov": trace_cv,
                            "days_forward": days_forward,
                            "dates_processed": 1,
                        })
                        mlflow.sklearn.log_model(
                            sk_model=model,
                            artifact_path="model",
                            registered_model_name=model_name,
                            input_example=input_example
                        )
                        mlflow.log_artifact(chart, artifact_path="charts")

                return None

            except Exception as e:
                return (asof_date, str(e))

        with ThreadPoolExecutor(max_workers=max_workers) as exe:
            futures = [exe.submit(task, d) for d in all_dates]
            for fut in as_completed(futures):
                if err := fut.result():
                    errors.append(err)

        mlflow.log_metrics({
            "dates_processed": len(all_dates) - len(errors),
            "n_errors": len(errors),
            "n_obs": sum(total_obs),
            "total_var": float(np.mean(total_vars)) if total_vars else 0.0,
            "trace_cov": float(np.mean(trace_covs)) if trace_covs else 0.0,
        })

        if errors:
            print(f"⚠️  {len(errors)} errors:")
            for d, msg in errors:
                print(f"  • {d}: {msg}")
        else:
            print("✅ All cones processed and logged.")


if __name__ == "__main__":
    import sys

    populate_ir_cones(
        backfill_days=backfill_DAYS,
        fit_window_years=1,
        max_workers=MAX_WORKERS,
        years_back=0
    )

    
    populate_ir_cones(
        backfill_days=backfill_DAYS,
        fit_window_years=5,
        max_workers=MAX_WORKERS,
        years_back=0
    )


'data_env' not found in environment. Defaulting to 'sandbox' env.
setting env to sandbox data
getting data source for sandbox


2025/06/24 17:23:42 INFO mlflow.tracking.fluent: Experiment with name 'IR Cone Fit Experiment' does not exist. Creating a new experiment.


⏭️  Skipping 2025-01-04: no exact curve_date in pivot
⏭️  Skipping 2024-12-28: no exact curve_date in pivot
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
⏭️  Skipping 2025-01-05: no exact curve_date in pivot
⏭️  Skipping 2024-12-29: no exact curve_date in pivot
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
⏭️  Skipping 2025-01-01: no exact curve_date in pivot
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
using model: GARCH(1,1)_1yrFit
⏭️  Skipping 2025-01-11: no exact curve_date in pivot
using model: GARCH(1,1)_1yrFit
⏭️  Skipping 2025-01-12: no exact curve_date in pivot
using model: GARCH(1,1)_1yrFit


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
/tmp/ipykernel_466/2289731576.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exists. Creating a new version of this model...
Registered model 'GARCH(1,1)_1yrFit' already exis

🏃 View run IR_2025-01-02 at: http://127.0.0.1:8768/#/experiments/1555/runs/8dc900ae2efa4bce88bc771c851330e9
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555
🏃 View run IR_2025-01-06 at: http://127.0.0.1:8768/#/experiments/1555/runs/76f6d381f7ad4905b062c439d7ff95b6
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555


...

🏃 View run IR_2025-04-28 at: http://127.0.0.1:8768/#/experiments/1555/runs/e35c1aa7bca641279dbb095a198f9fdc
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555


Registered model 'GARCH(1,1)_5yrFit' already exists. Creating a new version of this model...
2025/06/24 17:53:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GARCH(1,1)_5yrFit, version 289
Created version '289' of model 'GARCH(1,1)_5yrFit'.
2025/06/24 17:53:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GARCH(1,1)_5yrFit, version 290
Created version '290' of model 'GARCH(1,1)_5yrFit'.
2025/06/24 17:53:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GARCH(1,1)_5yrFit, version 291
Created version '291' of model 'GARCH(1,1)_5yrFit'.
Registered model 'GARCH(1,1)_5yrFit' already exists. Creating a new version of this model...
2025/06/24 17:53:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creatio

🏃 View run IR_2025-04-15 at: http://127.0.0.1:8768/#/experiments/1555/runs/572584aae45c41e68805b2dca0b3bad7
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555


2025/06/24 17:53:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GARCH(1,1)_5yrFit, version 293
Created version '293' of model 'GARCH(1,1)_5yrFit'.


🏃 View run IR_2025-04-25 at: http://127.0.0.1:8768/#/experiments/1555/runs/9092076f3935424f9b3b8af6069d7e5c
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555
🏃 View run IR_2025-04-29 at: http://127.0.0.1:8768/#/experiments/1555/runs/66913b95dfc840efaaf8edda0229897d
🏃 View run IR_2025-04-14 at: http://127.0.0.1:8768/#/experiments/1555/runs/e67b57481fac493db45d746fdf613427
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555
using model: GARCH(1,1)_5yrFit
🏃 View run IR_2025-04-11 at: http://127.0.0.1:8768/#/experiments/1555/runs/11fa233fedd0452c8432639bd1a876cf
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1555
using model: GARCH(1,1)_5yrFit
using model: GARCH(1,1)_5yrFit
